Universidad del Valle de Guatemala

Universidad del Valle de Guatemala

## Proyecto 1: Análisis exploratorio

Universidad del Valle de Guatemala
Laura Tamath	19365 |     Andrea Amaya 19357 |
Brandon Hernández	 19376 |		Martin Amado	19020 |   Juan Pablo Pineda 19087

In [ ]:
import pandas as pd 
import numpy as np
import pyreadstat
pd.options.display.max_columns = None

In [ ]:
var_names = ['dep_reg', 'mun_reg', 'mon_reg', 'year_reg', 'dep_occu','mun_occu','area_geog','sex_death','day_occu',
'month_occu','year_occu','part_type','birth_class','via_part','weeks_ges', 'mother_age' ,'mom_country_res','mom_dep_res',
'mom_mun_resi', 'mom_group', 'mom_civil_status', 'mom_nationality', 'mom_scholarship', 'mom_occupation', 'cause_death',
'assistance_received', 'site_occu', 'total_children', 'total_dead_children', 'total_living_children']

In [ ]:
def filter_df(data_frame, to_remove, is_quali=True):
  df = data_frame.copy()
  all_vars = var_names[:]
  remove_vars = to_remove[:]

  for var in remove_vars: all_vars.remove(var)
  df.columns = all_vars
  for var in remove_vars:
    df[var] = np.full(len(df.index), np.nan if is_quali else 0)
  return df.reindex(sorted(df.columns), axis=1).copy()

In [ ]:
df_2009 = pd.read_spss('./data/db_2009.sav')
df_2010 = pd.read_spss('./data/db_2010.sav')
df_2011 = pd.read_spss('./data/db_2011.sav')
df_2012 = pd.read_spss('./data/db_2012.sav')
df_2013 = pd.read_spss('./data/db_2013.sav')
df_2014 = pd.read_spss('./data/db_2014.sav')

# Filter data
remove_2009 = ['via_part', 'mom_country_res', 'mom_scholarship']
remove_2010_2011 = ['mom_country_res']
remove_2012_2013_2014 = ['year_occu']
df_2009 = filter_df(df_2009, remove_2009)
df_2010 = filter_df(df_2010, remove_2010_2011)
df_2011 = filter_df(df_2011, remove_2010_2011)
df_2012 = filter_df(df_2012, remove_2012_2013_2014)
df_2012['year_occu'] = np.full(len(df_2012.index), '2012')
df_2013 = filter_df(df_2013, remove_2012_2013_2014)
df_2013['year_occu'] = np.full(len(df_2013.index), '2013')
df_2014 = filter_df(df_2014, remove_2012_2013_2014)
df_2014['year_occu'] = np.full(len(df_2014.index), '2014')

In [ ]:
df_2015 = pd.read_spss('./data/db_2015.sav')
df_2016 = pd.read_spss('./data/db_2016.sav')
df_2017 = pd.read_spss('./data/db_2017.sav')
df_2018 = pd.read_spss('./data/db_2018.sav')
df_2019 = pd.read_spss('./data/db_2019.sav')
df_2020 = pd.read_spss('./data/db_2020.sav')
# Filter data
remove_2018_2019_2020 = ['area_geog']
df_2015= var_names[:]
df_2016= var_names[:]
df_2017= var_names[:]
df_2018 = filter_df(df_2018, remove_2018_2019_2020)
df_2019 = filter_df(df_2018, remove_2018_2019_2020)
df_2020 = filter_df(df_2018, remove_2018_2019_2020)

In [ ]:
data = pd.merge(df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020)
data

## Exploración de datos

1. Comience  describiendo  cuantas  variables  y  observaciones  tiene  disponibles,  el tipo de cada una de las variables. 

2. Haga un resumen de las variables numéricas e investigue si siguen una distribución normal y tablas de frecuencia para las variables categóricas, escriba lo que vaya encontrando. 

3. Cruce  las  variables  que  considere  que  son  las  más  importantes  para  hallar  los elementos  clave  que  lo  pueden  llevar  a  comprender  lo  que  está  causando  el problema encontrado.

4. Haga gráficos exploratorios que le de ideas del estado de los datos. 

5. Haga un agrupamiento (clustering) e interprete los resultados. 

## Luego de explorar los datos

1. Describa la situación problemática que lo lleva a acotar un problema a resolver.  

2. Enuncie un problema científico y unos objetivos preliminares.  

3. Describa  los  datos  que  tiene  para  responder  el  problema  planteado.  Esto  incluye el  estado  en  que  encontró  el  o  los  conjuntos  de  datos  y  las  operaciones  de limpieza que le realizó, en caso de que hayan sido necesarias.

4. Escriba unas conclusiones con los hallazgos encontrados durante el análisis 
exploratorio 